# Сервис для чтения книг по подписке.

**ЦЕЛЬ:**
— проанализировать базу данных и сформулировать ценностное предложение для нового продукта.

**Задачи:**
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

In [2]:
query = '''
        SELECT *
        FROM books
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` - Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Соеденина с таблицей **ratings** и **reviews** по book_id

In [3]:
query = '''
        SELECT*
        FROM ratings
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` - Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Соеденина с **books** и **reviews** по book_id

In [4]:
query = '''
        SELECT *
        FROM reviews
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` - Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Соединена с **books** и **ratings** по book_id

In [5]:
query = '''
        SELECT *
        FROM authors
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` - Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Соединена с таблицей **books** по author_id

In [6]:
query = '''
        SELECT *
        FROM publishers
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` - Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Соеденина с таблицей **books** по publisher_id

## Задание 1

**Посчитать, сколько книг вышло после 1 января 2000 года**

In [7]:
query = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01'
    '''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


- 819 книг вышло после 1 января 2000 года. Не с чем сравнивать, много это или мало

## Задание 2

**Для каждой книги посчитать количество обзоров и среднюю оценку**

In [8]:
query = '''
        WITH 
        s AS (SELECT book_id,
                          COUNT(review_id) AS count_review
                   FROM reviews AS re 
                   GROUP BY book_id),
        r AS (SELECT book_id,
                     AVG(rating) AS avg_rating
              FROM ratings
              GROUP BY book_id)
                   
        SELECT  s.book_id,
                b.title,
                r.avg_rating,
                s.count_review
        FROM s
        JOIN r ON r.book_id=s.book_id
        JOIN books AS b ON b.book_id = s.book_id
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,avg_rating,count_review
0,652,The Body in the Library (Miss Marple #3),4.500000,2
1,273,Galápagos,4.500000,2
2,51,A Tree Grows in Brooklyn,4.250000,5
3,951,Undaunted Courage: The Pioneering First Missio...,4.000000,2
4,839,The Prophet,4.285714,4


 - Сгруппировали таблицу по id, посчитали средний балл за книгу и кол-во обзоров. Можно посмотреть средний балл за книгу и выбрать те книги с наибольшим баллом, которые точно должны попасть в приложение

## Задание 3

**Определить издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [9]:
query = '''
        SELECT p.publisher_id, 
            COUNT(b.book_id) AS count_book, 
            p.publisher 
        FROM books AS b
        LEFT OUTER JOIN publishers AS p ON b.publisher_id=p.publisher_id
        WHERE b.num_pages > 50
        GROUP BY p.publisher_id
        ORDER BY count_book DESC
        LIMIT 5
    '''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,count_book,publisher
0,212,42,Penguin Books
1,309,31,Vintage
2,116,25,Grand Central Publishing
3,217,24,Penguin Classics
4,35,19,Bantam


- Penguin Books выпустили больше всего книг - 42. То издательство, с которым точно нужно сотрудничать

## Задание 4

**Определить автора с самой высокой средней оценкой книг**

In [10]:
query = '''
        WITH x AS (
            SELECT book_id, 
                AVG(rating) AS avg_rating,
                COUNT(book_id) AS count_review
            FROM ratings
            GROUP BY book_id
            ORDER BY count_review DESC)
        SELECT  a.author,
                AVG(x.avg_rating) AS avg_rating 
        FROM books as b
        LEFT OUTER JOIN authors AS a ON b.author_id=a.author_id
        LEFT OUTER JOIN x AS x ON x.book_id=b.book_id
        WHERE count_review > 50
        GROUP BY a.author
        ORDER BY avg_rating DESC
        LIMIT 5
                
    '''
pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


- Аавтор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4.28. Тот автор, чьи книги определенно должны быть в приложении. 

## Задача 5

**Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [11]:
query = '''
        WITH 
        data AS (SELECT username, 
                             COUNT(review_id) AS count_review
                        FROM reviews
                        GROUP BY username),
        rat AS (SELECT username,
                            COUNT(rating_id) AS count_rat
                     FROM ratings
                     GROUP BY username)
                     
        SELECT ROUND(AVG(d.count_review), 1)
        FROM data AS d
        LEFT OUTER JOIN rat AS r ON d.username = r.username
        WHERE r.count_rat > 50
        
    '''
pd.io.sql.read_sql(query, con = engine) 

,round
0,24.3


- Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3

## Вывод

- БД проанализированна. 
- Здания выполнены.

В ходе выполнения задания с целья получения ценностного предложения для нового приложения, получилась следущая информация:
- 819 книг вышло после 1 января 2000 года
- Возможно посмотреть средний балл за книгу и выбрать те книги с наибольшим баллом, которые точно должны попасть в приложение, благодаря новой сформированной таблице
- Penguin Books выпустили больше всего книг - 42. То издательство, с которым точно нужно сотрудничать
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4.28. Тот автор, чьи книги определенно должны быть в приложении.
- Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3